# Generate ML Pipeline at DevOps Build Pipeline

In the prevous section we learned how to create, publish and schedule an ML pipeline. But from the initial diagram (Displayed below) we want to generate an ML Pipeline when ever there is a new code in the master branch.

<img style="width:100%" src="assets/MLOpsArchFlow.jpg">

In this notebook, we aim to make some modifications to the previous notebook (MLPipeline_MNIST) so that Azure DevOps Build Pipeline can generate a new ML Pipeline every time the master branch of the GitHub repo is changed.

This is an important step to build a fully automated CI/CD pipeline for our ML project. So the scenario works like this:

As a new code hits the master branch (this time we like to trigger the build Pipeline at the CI "merge into the Master branch") that hosts our code for the training pipeline, we like to execute the code to generate a new ML Pipeline with the new code. The ML Pipeline then generates a new ML model. The ML models is evaluated and if the accuracy is higher than the existing model, it is pushed into production.

One major difference in this scenario is that we have to generate the ML Pipeline from the Ubunto computer within Azure DevOps. That computer doesn't have access to our Azure's subscription and also we don't want to manually go through the authentication process. We want this to be automatic. Therefore, we need to create a mechanisim that the machine can log in in absence of us to access our Azure environment and in particular our Azure Workspace.

One way to do this is to create a user name of type Service Principle. This user name is designed to let applications authenticate into Azure. So first we need to create a Service Principle Account. The steps are provided here: https://docs.microsoft.com/en-us/azure/active-directory/develop/howto-create-service-principal-portal

Save the following pieces of information: **Application ID**, **Tenant ID**, **Secret Key** and replace them in the code below:

1. Create an Azure Active Directory application
2. Assign the application to a role
3. Get values for signing in
4. Certificates and secrets -> Create a new application secret



**Important points from the previous session:**
* Pay attention to the requirements.txt as we only need to generate an ML Pipeline, azureml-sdk is the only package we need
* Pay attention to the tests folder. Make sure you have at least 1 test under tests folder.

Like always we import some packages related to the Azure ML:

In [3]:
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

print("Pipeline SDK-specific imports completed")

SDK version: 1.9.0
Pipeline SDK-specific imports completed


In [4]:
tenant_id = "e65922f9-4bd4-4f11-b1a3-48e89d75674e"
application_id = "423b1df1-5286-4955-a6e7-fc416aea35d4"
object_id = "28320c05-22bd-4b9b-b692-5e0da5fa69dc"
subscription_id = "b198933e-f055-498f-958d-0726ab11eddb"
app_secret = "v3d43JhbJ1x.A5mJ3hyS2GRD.O_ZC~FE6~"
resource_group = "MLOps_Template"
workspace_name = "MLOps_template_ML"
workspace_region = "West US 2"

Creation of Service Principal Identity makes the code capable of accessing to our Azure Environment and Access the ML Workspace. In this case, it can create the Pipeline through the Build Pipeline.

Now you download the file as Python and push the changes to the source repo. Now, we're ready to create our next gen Build Pipeline.

In [5]:
from azureml.core.authentication import ServicePrincipalAuthentication

service_principal = ServicePrincipalAuthentication(
        tenant_id=tenant_id,
        service_principal_id=application_id,
        service_principal_password=app_secret)

In [ ]:
#from azureml.core.authentication import InteractiveLoginAuthentication
#interactive_auth = InteractiveLoginAuthentication(tenant_id="e65922f9-4bd4-4f11-b1a3-48e89d75674e")

In [6]:
ws = Workspace.get(
            name=workspace_name,
            subscription_id=subscription_id,
            resource_group=resource_group,
            auth=service_principal)

In [7]:
# Retrieve the pointer to the default Blob storage.

def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))

Blobstore's name: workspaceblobstore


In [8]:
blob_input_data = DataReference(
    datastore=def_blob_store,
    data_reference_name="mnist_datainput",
    path_on_datastore="mnist_datainput")

print("DataReference object created")

DataReference object created


In [9]:
# Create a GPU cluster of type NV6 with 1 node. (due to subscription's limitations we stick to 1 node)

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpucluster"

try:
    compute_target_cpu = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    # CPU: Standard_D3_v2
    # GPU: Standard_NV6
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=1,
                                                           min_nodes=1)

    # create the cluster
    compute_target_cpu = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target_cpu.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target_cpu.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-07-23T21:23:31.815000+00:00', 'errors': None, 'creationTime': '2020-07-23T21:22:16.027354+00:00', 'modifiedTime': '2020-07-23T21:22:32.031336+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': ''}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [ ]:
# choose a name for your cluster
#cluster_name = "gpucluster"

#try:
#    compute_target_gpu = ComputeTarget(workspace=ws, name=cluster_name)
#    print('Found existing compute target.')
#except ComputeTargetException:
#    print('Creating a new compute target...')
#    # CPU: Standard_D3_v2
#    # GPU: Standard_NV6
#    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NV6', 
#                                                           max_nodes=1,
#                                                           min_nodes=1)

#    # create the cluster
#    compute_target_gpu = ComputeTarget.create(ws, cluster_name, compute_config)

#    compute_target_gpu.wait_for_completion(show_output=True)

## use get_status() to get a detailed status for the current cluster. 
#print(compute_target_gpu.get_status().serialize())

In [10]:
cts = ws.compute_targets
for ct in cts:
    print(ct)


cpucluster


In [11]:
processed_mnist_data = PipelineData("processed_mnist_data", datastore=def_blob_store)
processed_mnist_data

$AZUREML_DATAREFERENCE_processed_mnist_data

In [12]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk',
                                                                                          'numpy'])

In [13]:
# source directory
source_directory = 'DataExtraction'

extractDataStep = PythonScriptStep(
    script_name="extract.py", 
    arguments=["--output_extract", processed_mnist_data],
    outputs=[processed_mnist_data],
    compute_target=compute_target_cpu, 
    source_directory=source_directory,
    runconfig=run_config)

print("Data Extraction Step created")

Data Extraction Step created


In [14]:
from azureml.train.dnn import TensorFlow

source_directory = 'Training'
est = TensorFlow(source_directory=source_directory,
                 compute_target=compute_target_cpu,
                 entry_script='train.py', 
                 use_gpu=False, 
                 framework_version='1.13')


In [15]:
from azureml.pipeline.steps import EstimatorStep

model_name = "tf_mnist_pipeline_devops.model"
trainingStep = EstimatorStep(name="Training-Step",
                             estimator=est,
                             estimator_entry_script_arguments=["--input_data_location", processed_mnist_data,
                                                               '--batch-size', 50,
                                                               '--first-layer-neurons', 300,
                                                               '--second-layer-neurons', 100,
                                                               '--learning-rate', 0.01,
                                                               "--release_id", 0,
                                                               '--model_name', model_name],
                             runconfig_pipeline_params=None,
                             inputs=[processed_mnist_data],
                             compute_target=compute_target_cpu)

print("Model Training Step is Completed")

Model Training Step is Completed


In [16]:
# source directory
source_directory = 'RegisterModel'

modelEvalReg = PythonScriptStep(
    name="Evaluate and Register Model",
    script_name="evaluate_model.py", 
    arguments=["--release_id", 0,
               '--model_name', model_name],
    compute_target=compute_target_cpu, 
    source_directory=source_directory,
    runconfig=run_config)

modelEvalReg.run_after(trainingStep)
print("Model Evaluation and Registration Step is Created")

Model Evaluation and Registration Step is Created


In [17]:
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment
pipeline = Pipeline(workspace=ws, steps=[extractDataStep, trainingStep, modelEvalReg])
pipeline_run = Experiment(ws, 'MNIST-Model-Training-Build-CI').submit(pipeline)


WARNING - 'azureml-defaults' is not included in the pip package list in the environment definition in RunConfiguration. This may result in job failures during execution.
WARNING - 'azureml-defaults' is not included in the pip package list in the environment definition in RunConfiguration. This may result in job failures during execution.


Created step extract.py [5719721f][b7d35166-f8b3-40ce-b825-51f34cb68afe], (This step is eligible to reuse a previous run's output)
Created step Training-Step [4c45351c][fca13a57-0e20-4878-b34a-f25a73b6a2ad], (This step is eligible to reuse a previous run's output)
Created step Evaluate and Register Model [fed8d656][86efaffe-f2ff-4408-9ed6-432d5e3528f6], (This step is eligible to reuse a previous run's output)
Submitted PipelineRun 55cf6f2a-7d77-4967-a6d3-f18224469e81
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/MNIST-Model-Training-Build-CI/runs/55cf6f2a-7d77-4967-a6d3-f18224469e81?wsid=/subscriptions/b198933e-f055-498f-958d-0726ab11eddb/resourcegroups/MLOps_Template/workspaces/MLOps_template_ML


In [18]:
pipeline_run.wait_for_completion(show_output=True, raise_on_error=True)

PipelineRunId: 55cf6f2a-7d77-4967-a6d3-f18224469e81
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/MNIST-Model-Training-Build-CI/runs/55cf6f2a-7d77-4967-a6d3-f18224469e81?wsid=/subscriptions/b198933e-f055-498f-958d-0726ab11eddb/resourcegroups/MLOps_Template/workspaces/MLOps_template_ML
PipelineRun Status: Running


StepRunId: e3cf74bd-0b49-4999-b955-920f4e06659c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/MNIST-Model-Training-Build-CI/runs/e3cf74bd-0b49-4999-b955-920f4e06659c?wsid=/subscriptions/b198933e-f055-498f-958d-0726ab11eddb/resourcegroups/MLOps_Template/workspaces/MLOps_template_ML

StepRun(extract.py) Execution Summary
StepRun( extract.py ) Status: Finished
{'runId': 'e3cf74bd-0b49-4999-b955-920f4e06659c', 'target': 'cpucluster', 'status': 'Completed', 'startTimeUtc': '2020-07-23T22:40:31.513242Z', 'endTimeUtc': '2020-07-23T22:40:31.608308Z', 'properties': {'azureml.reusedrunid': 'f87bf8d0-ffd8-4939-88c5-9057334142dd', 'azur




StepRunId: 14c7d3ff-2528-49f5-b06c-a91d297e4cd4
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/MNIST-Model-Training-Build-CI/runs/14c7d3ff-2528-49f5-b06c-a91d297e4cd4?wsid=/subscriptions/b198933e-f055-498f-958d-0726ab11eddb/resourcegroups/MLOps_Template/workspaces/MLOps_template_ML
StepRun( Training-Step ) Status: NotStarted
StepRun( Training-Step ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_62442e26556e55e1e78592cd3ee7559fc46ef53f85f9ec408801ba7fa845ab4e_d.txt
2020-07-23T22:40:48Z Executing 'Copy ACR Details file' on 10.0.0.4
2020-07-23T22:40:48Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
2020-07-23T22:40:48Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2020-07-23T22:40:48Z Starting output-watcher...
Login Succeeded
1.13-cpu: Pulling from tensorflow
16c48d79e9cc: Pulling fs layer
3c654ad3ed7d: Pulling fs layer
6276f4f9c29d: Pulling fs layer
a4bd43ad48ce: Pulling fs layer
e8d5220518a8: Pulling fs lay

15 -- Training accuracy: 0.96 Validation accuracy: 0.9681
16 -- Training accuracy: 0.98 Validation accuracy: 0.9692
17 -- Training accuracy: 0.96 Validation accuracy: 0.9698
18 -- Training accuracy: 1.0 Validation accuracy: 0.9706
19 -- Training accuracy: 1.0 Validation accuracy: 0.9718

Streaming azureml-logs/75_job_post-tvmps_62442e26556e55e1e78592cd3ee7559fc46ef53f85f9ec408801ba7fa845ab4e_d.txt
Entering job release. Current time:2020-07-23T22:43:19.875860
Starting job release. Current time:2020-07-23T22:43:20.759569
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 261
[2020-07-23T22:43:20.773916] Entering context manager injector.
Job release is complete. Current time:2020-07-23T22:43:21.912205

StepRun(Training-Step) Execution Summary
StepRun( Training-Step ) Status: Finished
{'runId': '14c7d3ff-2528-49f5-b06c-a91d297e4cd4', 'target': 'cpucluster', 'status': 'Completed', 'startTimeUtc': '2020-

StepRun( Evaluate and Register Model ) Status: NotStarted
StepRun( Evaluate and Register Model ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_62442e26556e55e1e78592cd3ee7559fc46ef53f85f9ec408801ba7fa845ab4e_d.txt
2020-07-23T22:44:17Z Executing 'Copy ACR Details file' on 10.0.0.4
2020-07-23T22:44:17Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
2020-07-23T22:44:17Z Starting output-watcher...
2020-07-23T22:44:17Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_2e1aa647468113323a439077734657c5
Digest: sha256:5ad4c6cc73968982b57bceab23f46f13cfabb51a13e0184bea30c22104c89a53
Status: Image is up to date for mlopstemplat9c2637c6.azurecr.io/azureml/azureml_2e1aa647468113323a439077734657c5:latest
2020-07-23T22:44:18Z 6cd1e7856fc05b8cd998a8903da216d07e57f336f03de263e6fe0663c12bf8f2
2020-07-23T22:44:18Z 
2020/07/23 22:44:19 Starting App Insight Logger for task:  contai


Streaming azureml-logs/75_job_post-tvmps_62442e26556e55e1e78592cd3ee7559fc46ef53f85f9ec408801ba7fa845ab4e_d.txt
Entering job release. Current time:2020-07-23T22:44:37.502582
Starting job release. Current time:2020-07-23T22:44:39.137525
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 159
[2020-07-23T22:44:39.158635] Entering context manager injector.
Job release is complete. Current time:2020-07-23T22:44:40.544922

StepRun(Evaluate and Register Model) Execution Summary
StepRun( Evaluate and Register Model ) Status: Finished
{'runId': 'cec16254-b7b2-4c5f-a614-1785d4fa3b9d', 'target': 'cpucluster', 'status': 'Completed', 'startTimeUtc': '2020-07-23T22:44:17.194739Z', 'endTimeUtc': '2020-07-23T22:44:52.246634Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': '53e8702f-c145-4f35-b415-c0eccaee5c02', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azurem

'Finished'

In [19]:
published_pipeline = pipeline_run.publish_pipeline(name="MNIST-Pipeline-Created-At-Build-Pipeline", 
                                                   description="Steps are: data preparation, training, model validation and model registration", 
                                                   version="0.1", 
                                                   continue_on_step_failure=False)